In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [2]:
dataset=pd.read_csv("DataCoSupplyChainDataset.csv",header= 0,encoding= 'unicode_escape')
dataset.head()

,Type,Days for shipping (real),Days for shipment (scheduled),Benefit per order,Sales per customer,Delivery Status,Late_delivery_risk,Category Id,Category Name,Customer City,...,Order Zipcode,Product Card Id,Product Category Id,Product Description,Product Image,Product Name,Product Price,Product Status,shipping date (DateOrders),Shipping Mode
0,DEBIT,3,4,91.250000,314.640015,Advance shipping,0,73,Sporting Goods,Caguas,...,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,2/3/2018 22:56,Standard Class
1,TRANSFER,5,4,-249.089996,311.359985,Late delivery,1,73,Sporting Goods,Caguas,...,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,1/18/2018 12:27,Standard Class
2,CASH,4,4,-247.779999,309.720001,Shipping on time,0,73,Sporting Goods,San Jose,...,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,1/17/2018 12:06,Standard Class
3,DEBIT,3,4,22.860001,304.809998,Advance shipping,0,73,Sporting Goods,Los Angeles,...,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,1/16/2018 11:45,Standard Class
4,PAYMENT,2,4,134.210007,298.250000,Advance shipping,0,73,Sporting Goods,Caguas,...,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,1/15/2018 11:24,Standard Class


In [3]:
# Adding first name and last name together to create new column
dataset['Customer Full Name'] = dataset['Customer Fname'].astype(str)+dataset['Customer Lname'].astype(str)
data=dataset.drop(['Customer Email','Product Status','Customer Password','Customer Street','Customer Fname','Customer Lname',
           'Latitude','Longitude','Product Description','Product Image','Order Zipcode','shipping date (DateOrders)'],axis=1)

In [4]:
data['Customer Zipcode']=data['Customer Zipcode'].fillna(0)

In [5]:
def outlier_treatment(datacolumn):
 sorted(datacolumn)
 Q1,Q3 = np.percentile(datacolumn , [25,75])
 IQR = Q3 - Q1
 lower_range = Q1 - (1.5 * IQR)
 upper_range = Q3 + (1.5 * IQR)
 return lower_range,upper_range

In [6]:
lower_range,upper_range=outlier_treatment(data['Product Price'])

In [7]:
data.drop(data[(data['Product Price'] < lower_range) | (data['Product Price'] > upper_range)].index , inplace=True)

In [8]:
train_data=data.copy()

In [9]:
train_data['fraud'] = np.where(train_data['Order Status'] == 'SUSPECTED_FRAUD', 1, 0)
train_data['late_delivery']=np.where(train_data['Delivery Status'] == 'Late delivery', 1, 0)

In [10]:
#Dropping columns with repeated values
train_data.drop(['Delivery Status','Late_delivery_risk','Order Status',
                # 'order_month_year',
                 'order date (DateOrders)'], axis=1, inplace=True)

In [11]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
train_data['Order Country']  = le.fit_transform(train_data['Order Country'])
train_data['Order State']    = le.fit_transform(train_data['Order State'])

In [12]:
Xf=train_data[['Days for shipping (real)','Days for shipment (scheduled)','Order Country']]
yf=train_data['fraud']

In [13]:
train_x,test_x,train_y,test_y = train_test_split(Xf,yf,test_size = 0.2, random_state = 42)

In [14]:
random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(train_x, train_y.values.ravel())
random_forest.score(train_x, train_y)
acc_random_forest = round(random_forest.score(train_x, train_y) * 100, 2)
acc_random_forest

97.81

In [25]:
import tabpy_client
from tabpy.tabpy_tools.client import Client
client = tabpy_client.Client('http://localhost:9004/')

def fraud_predictor5( _arg1, _arg2,_arg3):
    import pandas as pd
    row = {'shipping': _arg1,
           'shipping scheduled': _arg2,
          'country_str':_arg3}
    #Convert it into a dataframe
    test_data = pd.DataFrame(data = row,index=[0])
    from sklearn import preprocessing
    le = preprocessing.LabelEncoder()
    test_data['country_str']  = le.fit_transform(test_data['country_str'])
    #Predict the survival and death probabilities
    predprob_survival = random_forest.predict_proba(test_data)
    #Return only the survival probability
    return [probability[1] for probability in predprob_survival]

In [ ]:
client.deploy('fraud_predictor5', fraud_predictor5,'fraud_predictor probability',override = True)

In [28]:
Xl=train_data[['Days for shipment (scheduled)','Order Country']]
yl=train_data['late_delivery']

In [29]:
train_xl,test_xl,train_yl,test_yl = train_test_split(Xl,yl,test_size = 0.2, random_state = 42)

In [31]:
random_forest_l = RandomForestClassifier(n_estimators=100)
random_forest_l.fit(train_xl, train_yl.values.ravel())
random_forest_l.score(train_xl, train_yl)
acc_random_forest = round(random_forest_l.score(train_xl, train_yl) * 100, 2)
acc_random_forest

69.72

In [32]:
import tabpy_client
from tabpy.tabpy_tools.client import Client
client = tabpy_client.Client('http://localhost:9004/')

def late_delivery( _arg1, _arg2):
    import pandas as pd
    row = {'shipping scheduled': _arg1,
          'country_str':_arg2}
    #Convert it into a dataframe
    test_data = pd.DataFrame(data = row,index=[0])
    from sklearn import preprocessing
    le = preprocessing.LabelEncoder()
    test_data['country_str']  = le.fit_transform(test_data['country_str'])
    #Predict the survival and death probabilities
    predprob_late = random_forest_l.predict_proba(test_data)
    #Return only the survival probability
    return [probability[1] for probability in predprob_late]

In [35]:
client.deploy('late_delivery', late_delivery,'late_delivery_prop',override = True)